# PRETRAINED NET WITH TF-SLIM

In [1]:
import urllib, os
import tarfile
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import resnet_v1
print ("PACKAGES LOADED")

PACKAGES LOADED


## DOWNLOAD PRETRAINED NET

In [2]:
# MAKE PATH IF NECESSARY 
dirpath = "data/"
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
# DOWNLOAD 
urlpath = "http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz"
zname   = "data/resnet_v1_50_2016_08_28.tar.gz"
if not os.path.exists(zname):
    print ("DOWNLOADING %s..." % (zname))
    urllib.urlretrieve(urlpath, zname)
    print ("DONE")
else:
    print ("%s ALREADY EXISTS" % (zname))

data/resnet_v1_50_2016_08_28.tar.gz ALREADY EXISTS


## UNZIP CKPT

In [3]:
fname = "data/resnet_v1_50_2016_08_28"
if not os.path.exists(fname):
    print ("EXTRACTING %s..." % (fname))
    tar = tarfile.open(zname, "r:gz")
    tar.extractall(path=fname)
    tar.close()
    print ("DONE.")
else:
    print ("%s ALREADY EXIST" % (fname))

data/resnet_v1_50_2016_08_28 ALREADY EXIST


## OPEN SESSION

In [4]:
sess = tf.Session()

## MODEL

In [5]:
x = tf.placeholder(shape=[None, 224, 224, 3], dtype=tf.float32)
# inputs has shape [batch, 224, 224, 3]
with slim.arg_scope(resnet_v1.resnet_arg_scope()):
    mean = tf.constant([123.68, 116.779, 103.939],
        dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
    im_centered = x - mean
    net, end_points = resnet_v1.resnet_v1_50(inputs=im_centered, num_classes=1000)
b1 = end_points['resnet_v1_50/block1']

## RESTORE

In [6]:
variables_to_restore = slim.get_variables_to_restore(include=["resnet_v1"])
restorer = tf.train.Saver(variables_to_restore)
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
restorer.restore(sess, fname+"/resnet_v1_50.ckpt") 

## RUN

In [7]:
currimg  = imread("data/cat.jpeg")
rszimg   = imresize(currimg, [224, 224])
rszimg   = rszimg.reshape([1, 224, 224, 3])
out      = sess.run(b1, feed_dict={x: rszimg})

In [8]:
print out.shape

(1, 28, 28, 256)
